In [ ]:
# api py:

from flask import Flask, request, jsonify, send_file, render_template
from flask_cors import CORS
import re
from io import BytesIO

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

import matplotlib.pyplot as plt
import pandas as pd
import pickle
import base64

app = Flask(__name__, template_folder="templates", static_folder="static")
CORS(app)

STOPWORDS = set(stopwords.words("english"))
stemmer = PorterStemmer()

predictor = pickle.load(open("./model_xgb.pkl", "rb"))
scaler = pickle.load(open("./scaler.pkl", "rb"))
cv = pickle.load(open("./countVectorizer.pkl", "rb"))

print("Model classes_:", getattr(predictor, "classes_", None))


@app.route("/test", methods=["GET"])
def test():
    return "Test request received successfully. Service is running"


@app.route("/", methods=["GET"])
def home():
    return render_template("landing.html")


@app.route("/predict", methods=["POST"])
def predict():
    try:
        # BULK prediction (CSV)
        if "file" in request.files and request.files["file"].filename != "":
            file = request.files["file"]
            data = pd.read_csv(file)

            predictions_csv, graph = bulk_prediction(predictor, scaler, cv, data)

            response = send_file(
                predictions_csv,
                mimetype="text/csv",
                as_attachment=True,
                download_name="Predictions.csv",
            )

            response.headers["X-Graph-Exists"] = "true"
            response.headers["X-Graph-Data"] = base64.b64encode(graph.getbuffer()).decode("ascii")

            return response

        # SINGLE prediction (JSON)
        elif request.is_json:
            payload = request.get_json(silent=True) or {}
            text_input = payload.get("text", "")
            if not text_input.strip():
                return jsonify({"error": "Missing 'text' field"}), 400

            predicted_sentiment = single_prediction(predictor, scaler, cv, text_input)
            return jsonify({"prediction": predicted_sentiment})

        return jsonify({"error": "Send either a CSV file OR JSON {'text': '...'}"}), 400

    except Exception as e:
        return jsonify({"error": str(e)}), 500


def clean_text(text: str) -> str:
    review = re.sub(r"[^a-zA-Z]", " ", str(text))
    words = review.lower().split()
    words = [stemmer.stem(word) for word in words if word not in STOPWORDS]
    return " ".join(words)


def single_prediction(predictor, scaler, cv, text_input: str) -> str:
    corpus = [clean_text(text_input)]
    X = cv.transform(corpus).toarray()

    nonzero = int((X != 0).sum())
    if nonzero == 0:
        return "Unknown (no recognized words in vocabulary)"

    X_scl = scaler.transform(X)

    proba = predictor.predict_proba(X_scl)[0]
    classes = list(getattr(predictor, "classes_", [0, 1]))  # ✅ CHANGED

    idx = int(proba.argmax())
    pred_class = classes[idx]

    # debug prints
    print("---- SINGLE PRED ----")
    print("RAW:", text_input)
    print("CLEAN:", corpus[0])
    print("NONZERO:", nonzero)
    print("CLASSES:", classes)
    print("PROBA:", proba, "=> pred_class:", pred_class)

    try:
        pred_int = int(pred_class)
    except Exception:
        s = str(pred_class).lower()
        if "pos" in s:
            return "Positive Sentiment"
        if "neg" in s:
            return "Negative Sentiment"
        return f"Prediction: {pred_class}"

    # ✅ FIXED MAPPING: 1=Positive, 0=Negative
    return "Positive Sentiment" if pred_int == 1 else "Negative Sentiment"


def bulk_prediction(predictor, scaler, cv, data: pd.DataFrame):
    if "Sentence" not in data.columns:
        raise ValueError("CSV must contain a column named 'Sentence'.")

    corpus = [clean_text(s) for s in data["Sentence"].astype(str).tolist()]
    X = cv.transform(corpus).toarray()
    X_scl = scaler.transform(X)

    proba = predictor.predict_proba(X_scl)
    classes = list(getattr(predictor, "classes_", [0, 1]))  # ✅ CHANGED
    preds = [classes[int(i)] for i in proba.argmax(axis=1)]  # ✅ CHANGED

    mapped = []
    for p in preds:
        try:
            pi = int(p)
            # ✅ FIXED MAPPING: 1=Positive, 0=Negative
            mapped.append("Positive" if pi == 1 else "Negative")
        except Exception:
            s = str(p).lower()
            if "pos" in s:
                mapped.append("Positive")
            elif "neg" in s:
                mapped.append("Negative")
            else:
                mapped.append(str(p))

    out = data.copy()
    out["Predicted sentiment"] = mapped

    predictions_csv = BytesIO()
    out.to_csv(predictions_csv, index=False)
    predictions_csv.seek(0)

    graph = get_distribution_graph(out)
    return predictions_csv, graph


def get_distribution_graph(data: pd.DataFrame) -> BytesIO:
    fig = plt.figure(figsize=(5, 5))
    tags = data["Predicted sentiment"].value_counts()

    tags.plot(
        kind="pie",
        autopct="%1.1f%%",
        shadow=True,
        startangle=90,
        title="Sentiment Distribution",
        xlabel="",
        ylabel="",
    )

    graph = BytesIO()
    plt.savefig(graph, format="png", bbox_inches="tight")
    plt.close(fig)
    graph.seek(0)
    return graph


if __name__ == "__main__":
    app.run(host="127.0.0.1", port=5000, debug=True)















## main.py: 

import streamlit as st
import pandas as pd
import requests
from io import BytesIO
import base64  # ✅ ADDED

prediction_endpoint = "http://127.0.0.1:5000/predict"  # ✅ CHANGED (was 5500)

st.title("Text Sentiment Predictor")

uploaded_file = st.file_uploader("Choose a CSV file for bulk prediction", type="csv")
user_input = st.text_input("Enter text for prediction", "")

if st.button("Predict"):
    if uploaded_file is not None:
        files = {"file": uploaded_file}
        res = requests.post(prediction_endpoint, files=files)

        if res.status_code != 200:
            st.error(f"Error {res.status_code}: {res.text}")
        else:
            df = pd.read_csv(BytesIO(res.content))
            st.dataframe(df.head(20))

            st.download_button(
                label="Download Predictions",
                data=res.content,
                file_name="Predictions.csv",
                mime="text/csv",
            )

            # ✅ ADDED: show pie chart if provided
            if res.headers.get("X-Graph-Exists") == "true":
                graph_b64 = res.headers.get("X-Graph-Data")
                if graph_b64:
                    st.image(BytesIO(base64.b64decode(graph_b64)))

    else:
        # ✅ CHANGED: send JSON (your Flask expects JSON)
        res = requests.post(prediction_endpoint, json={"text": user_input})

        if res.status_code != 200:
            st.error(f"Error {res.status_code}: {res.text}")
        else:
            data = res.json()
            st.write(f"Predicted sentiment: {data.get('prediction')}")




# index.html

# <!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1.0" />
  <title>Amazon Review Sentiment</title>
</head>
<body>
  <h1>Predicting text sentiment</h1>

  <form onsubmit="return false;">
    <input type="file" id="csvFileInput" accept=".csv">
    <textarea id="textInput" placeholder="Enter text..."></textarea>

    <button type="button" id="predictBtn">Predict</button> <!-- ✅ CHANGED -->
    <button type="button" id="downloadBtn" style="display:none;">Download Predictions</button> <!-- ✅ CHANGED -->
  </form>

  <div id="predictionResult"></div>
  <div id="graphContainer"></div>

  <script>
    const API = "http://127.0.0.1:5000"; // ✅ CHANGED

    document.getElementById("predictBtn").addEventListener("click", predict); // ✅ ADDED

    async function predict() {
      const csvFileInput = document.getElementById("csvFileInput");
      const textInput = document.getElementById("textInput");
      const predictionResult = document.getElementById("predictionResult");
      const graphContainer = document.getElementById("graphContainer");
      const downloadBtn = document.getElementById("downloadBtn");

      predictionResult.textContent = "";
      graphContainer.innerHTML = "";
      downloadBtn.style.display = "none";
      downloadBtn.onclick = null;

      try {
        if (csvFileInput.files.length > 0) {
          const formData = new FormData();
          formData.append("file", csvFileInput.files[0]);

          const res = await fetch(`${API}/predict`, { method: "POST", body: formData });
          const blob = await res.blob();

          if (!res.ok) throw new Error(await blob.text());

          if (res.headers.get("X-Graph-Exists") === "true") {
            const graphData = res.headers.get("X-Graph-Data");
            if (graphData) displayGraph(graphData);
          }

          downloadBtn.style.display = "inline-block";
          downloadBtn.onclick = () => {
            const url = URL.createObjectURL(blob);
            const a = document.createElement("a");
            a.href = url;
            a.download = "Predictions.csv";
            document.body.appendChild(a);
            a.click();
            a.remove();
            URL.revokeObjectURL(url);
          };

          predictionResult.textContent = "Bulk prediction ready. Download CSV.";

        } else if (textInput.value.trim() !== "") {
          const res = await fetch(`${API}/predict`, {
            method: "POST",
            headers: { "Content-Type": "application/json" },
            body: JSON.stringify({ text: textInput.value.trim() })
          });

          const t = await res.text();
          if (!res.ok) throw new Error(t);

          const data = JSON.parse(t);
          predictionResult.textContent = "Predicted sentiment: " + data.prediction;

        } else {
          predictionResult.textContent = "Please upload a CSV or enter text.";
        }
      } catch (e) {
        predictionResult.textContent = "Error: " + e.message;
        console.error(e);
      }
    }

    function displayGraph(graphData) {
      const graphContainer = document.getElementById("graphContainer");
      graphContainer.innerHTML = "";
      const img = document.createElement("img");
      img.src = "data:image/png;base64," + graphData;
      graphContainer.appendChild(img);
    }
  </script>
</body>
</html>














# landing.html

<!DOCTYPE html>
<html>
<head>
  <meta charset="utf-8" />
  <meta name="viewport" content="width=device-width, initial-scale=1" />
  <link rel="stylesheet" href="https://cdn.jsdelivr.net/gh/creativetimofficial/tailwind-starter-kit/compiled-tailwind.min.css" />
  <title>Amazon Alexa Review Analysis</title>
</head>

<body class="text-gray-800 antialiased">
  <main>
    <section class="py-20">
      <div class="container mx-auto px-4 py-8">
        <div class="items-center flex flex-wrap">

          <div class="w-full md:w-4/12 ml-auto mr-auto px-4">
            <!-- ✅ CHANGED: ONLY ONE FORM (no nesting), no duplicate ids -->
            <form id="predictionForm" onsubmit="return false;">
              <div class="mx-auto max-w-lg">
                <div class="py-2">
                  <span class="px-1 text-sm text-gray-600">Upload your CSV file</span>
                  <input type="file" id="csvFileInput" accept=".csv"
                    class="text-md block px-3 py-2 rounded-lg w-full bg-white border-2 border-gray-300 shadow-md">
                </div>

                <div class="py-2">
                  <span class="px-1 text-sm text-gray-600">Text for Prediction</span>
                  <textarea id="textInput" placeholder="Enter text..."
                    class="text-md block px-3 py-2 rounded-lg w-full bg-white border-2 border-gray-300 shadow-md"></textarea>
                </div>

                <!-- ✅ CHANGED: no inline onclick; attach listener in JS -->
                <button type="button" id="predictBtn"
                  class="mt-3 text-lg font-semibold bg-gray-800 w-full text-white rounded-lg px-6 py-3 shadow-xl hover:bg-black">
                  Predict
                </button>
              </div>
            </form>
          </div>

          <div class="w-full md:w-4/12 ml-auto mr-auto px-4">
            <h1 class="text-2xl font-semibold">Prediction Result</h1>
            <div class="p-4 m-2 border">
              <div id="predictionResult"></div>
            </div>

            <h1 class="text-2xl font-semibold mt-6">Graph Result</h1>
            <div class="p-4 m-2 border">
              <div id="graphContainer"></div>
            </div>

            <!-- ✅ CHANGED: type="button" -->
            <button id="downloadBtn" type="button" style="display:none"
              class="text-lg font-semibold bg-gray-800 w-full text-white rounded-lg px-6 py-3 mt-6 shadow-xl hover:bg-black">
              Download Predictions
            </button>
          </div>

        </div>
      </div>
    </section>
  </main>

  <script>
    const API = "http://127.0.0.1:5000"; // ✅ CHANGED: correct backend port

    document.getElementById("predictBtn").addEventListener("click", predict); // ✅ ADDED

    async function predict() {
      const csvFileInput = document.getElementById("csvFileInput");
      const textInput = document.getElementById("textInput");
      const predictionResult = document.getElementById("predictionResult");
      const graphContainer = document.getElementById("graphContainer");
      const downloadBtn = document.getElementById("downloadBtn");

      predictionResult.textContent = "";
      graphContainer.innerHTML = ""; // ✅ ADDED: clear old graph
      downloadBtn.style.display = "none";
      downloadBtn.onclick = null;

      try {
        if (csvFileInput.files.length > 0) {
          const formData = new FormData();
          formData.append("file", csvFileInput.files[0]);

          const res = await fetch(`${API}/predict`, { method: "POST", body: formData });
          const bodyBlob = await res.blob();

          if (!res.ok) {
            const errText = await bodyBlob.text();
            throw new Error(errText);
          }

          // ✅ CHANGED: read graph headers
          if (res.headers.get("X-Graph-Exists") === "true") {
            const graphData = res.headers.get("X-Graph-Data");
            if (graphData) displayGraph(graphData);
          }

          downloadBtn.style.display = "block";
          downloadBtn.onclick = function () {
            const url = URL.createObjectURL(bodyBlob);
            const a = document.createElement("a");
            a.href = url;
            a.download = "Predictions.csv";
            document.body.appendChild(a);
            a.click();
            a.remove();
            URL.revokeObjectURL(url);
          };

          predictionResult.textContent = "Bulk prediction complete. Download ready.";

        } else if (textInput.value.trim() !== "") {
          const res = await fetch(`${API}/predict`, {
            method: "POST",
            headers: { "Content-Type": "application/json" },
            body: JSON.stringify({ text: textInput.value.trim() })
          });

          const text = await res.text(); // ✅ ADDED: better error visibility
          if (!res.ok) throw new Error(text);

          const data = JSON.parse(text);
          predictionResult.textContent = "Predicted sentiment: " + data.prediction;

        } else {
          predictionResult.textContent = "Please upload a CSV or enter text.";
        }
      } catch (err) {
        predictionResult.textContent = "Error: " + err.message; // ✅ ADDED
        console.error(err);
      }
    }

    function displayGraph(graphData) {
      const graphContainer = document.getElementById("graphContainer");
      graphContainer.innerHTML = "";
      const img = document.createElement("img");
      img.src = "data:image/png;base64," + graphData;
      graphContainer.appendChild(img);
    }
  </script>
</body>
</html>